In [49]:
!pip install import-ipynb
# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By

import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select

#installation guide for selenium
#https://medium.com/@shanyitan/how-to-install-selenium-and-run-it-successfully-via-jupyter-lab-c3f50d22a0d4

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2979 sha256=005aa82bf8ca8c045e446f83f95950d26a7bce6da61486ee58802805631998ea
  Stored in directory: c:\users\tomer\appdata\local\pip\cache\wheels\b1\5e\dc\79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


#### regexing

In [51]:
def data_gatherer(scraped_data):
    """ Return (dictionary that contains all the courses per page, canceled courses)"""
    database = dict()
    canceled_courses = []
    for course in scraped_data:
        course_details = []
        info_by_day = dict()
        try:
            name=re.findall(r"(\n[a-z,A-Z]\D{2,}\|)", course)[0].split("|")
            name = re.sub(r"[\n\t]*", "", name[0]).replace("  "," ")

        except:
            name = 'no name'
            
        hours = (re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", course))
        course_id = (re.findall(r"\|\s{3}(\d{5})", course))
        day = (re.findall(r"\n(יום [אבגדהו])", course))
        semester = (re.findall(r"\n(שנתי|סמסטר [אבגדהו])", course))
          
        #checking for canceled courses
        if len(course_id) == 0 or len(hours) == 0:
            if len(hours) == 0:
                canceled_courses.append([course_id,name])
            pass

        else:
            for item in range(len(semester)):
                info_by_day={}
                info_by_day["semester"] = semester[item] 
                try:
                    info_by_day["day"] = day[item]
                except:
                    info_by_day["day"] = 'None'
                try:
                    info_by_day["hours"] = hours[item]
                except:
                    info_by_day["hours"] = 'None' 
                course_details.append(info_by_day)
            database[course_id[0]] = {name:course_details}

    canceled_courses = [x for x in canceled_courses if x != []]
    return database, canceled_courses
database, canceled_courses = data_gatherer(scraped_data)

In [56]:
#sorting the courses dict

database_sorted = {}
for key, value in sorted(database.items()): # Note the () after items!
    database_sorted[key] = value

#removing duplicates and sorting removed
dicto = {}
for course in canceled_courses:
    dicto[str(course)] = None
canceled_sorted=sorted((dicto))
print("\n\n number of active courses in updated scraper:",len(database_sorted.keys()))
print("\n\n list of "+str(len(canceled_sorted)))





 number of active courses in updated scraper: 4712


 list of 1309
